In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import Perceptron
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression

%matplotlib inline

In [2]:
test_filename = 'test.csv'
train_filename = 'train.csv'
test_df = pd.read_csv(test_filename)
train_df = pd.read_csv(train_filename)

# Remove the first column
np_train_df = train_df.to_numpy()[:, 1:]
np_test_df = test_df.to_numpy()[:, 1:]

# CONSTRAINTS
DIGIT = 0
FIRST_PIXEL = 1

### Funções utilitárias

In [3]:
def pos_pixel(size, i ,j):
  return i*size + j

def symmetry(image):
  size = 28
  sv = 0
  sh = 0
  
  for i in range(size):
    for j in range(int(size/2)):
      p1 = image[pos_pixel(size, i, j)]
      p2 = image[pos_pixel(size, i, (size-1)-j)]
      sv += abs(p1 - p2)
            
      p1 = image[pos_pixel(size, j, i)]
      p2 = image[pos_pixel(size, (size-1)-j, i)]
      sh += abs(p1 - p2)

  sv = sv/255
  sh = sh/255
        
  #s = sv
  #s = sh
  #s = pow(sv + sh, 2)
  s = sv + sh
  #s = abs(sv - sh)
  
  return s
  
def intensity(image):
  return (np.sum(image)/255.0)


def get_numbers_1_and_5(df):
  filtered_data = []

  for item in df:
    if(item[0] == 1 or item[0] == 5):
      filtered_data.append(item.tolist())

  return filtered_data
  

### Classificação dos números 1 e 5

Dados de treinamento

In [21]:
new_train_df = []
images = get_numbers_1_and_5(np_train_df)

for image in images:
  symmetry_ = symmetry(image[FIRST_PIXEL:])
  intensity_ = intensity(image[FIRST_PIXEL:])

  new_train_df.append([image[DIGIT], symmetry_, intensity_])

Dados de teste

In [22]:
new_test_df = []
images = get_numbers_1_and_5(np_test_df)

for image in images:
  symmetry_ = symmetry(image[FIRST_PIXEL:])
  intensity_ = intensity(image[FIRST_PIXEL:])

  new_test_df.append([image[DIGIT], symmetry_, intensity_])

### Classificação de todos os dígitos

Dados de treinamento

In [21]:
new_train_df = []

for image in np_train_df:
  symmetry_ = symmetry(image[FIRST_PIXEL:])
  intensity_ = intensity(image[FIRST_PIXEL:])

  new_train_df.append([image[DIGIT], symmetry_, intensity_])

new_train_df = np.array(new_train_df)

Dados de teste

In [22]:
new_test_df = []

for image in np_test_df:
  symmetry_ = symmetry(image[FIRST_PIXEL:])
  intensity_ = intensity(image[FIRST_PIXEL:])

  new_test_df.append([image[DIGIT], symmetry_, intensity_])

new_test_df = np.array(new_test_df)

In [26]:
# PERCEPTRON
pla_model = Perceptron(shuffle=True, random_state=42)
pla_model.fit(new_train_df, new_train_df[:, :1].ravel())

# LINEAR REGRESSION

# LOGISTIC REGRESSION

<bound method ClassifierMixin.score of Perceptron(random_state=42)>

In [ ]:
new_train_df = []
new_test_df = []

for row in train_df.iterrows():
    data_row = row[1]

    id = data_row[0]
    label = data_row[1]
    imagem = data_row[2:]

    intensity_ = intensity(imagem)
    symmetry_ = symmetry(imagem)
    new_train_df.append([label,intensity_,symmetry_])

for row in test_df.iterrows():
    data_row = row[1]

    id = data_row[0]
    label = data_row[1]
    imagem = data_row[2:]

    intensity_ = intensity(imagem)
    symmetry_ = symmetry(imagem)
    new_test_df.append([label,intensity_,symmetry_])

In [4]:
new_train_df = pd.DataFrame(new_train_df, columns=['label', 'intensity', 'symmetry'])
new_test_df = pd.DataFrame(new_test_df, columns=['label', 'intensity', 'symmetry'])

In [5]:
new_test_df.to_csv('test_redu.csv', index= False)
new_train_df.to_csv('train_redu.csv', index= False)